<a href="https://colab.research.google.com/github/Noodle-bg/F.R.I.E.N.D.S-GPT/blob/main/Baby_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
with open('Friends_script.txt','r',encoding='utf-8') as f:
  text = f.read()

In [6]:
print(len(text))

4176958


In [7]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

	
 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_`abcdefghijklmnopqrstuvwxyz{|}¦­Éâäçèéíī–—‘’“”…€™�
114


In [8]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l:''.join([itos[i] for i in l])


print(encode("Hello world"))
print(decode(encode("Hello world")))

[42, 69, 76, 76, 79, 2, 87, 79, 82, 76, 68]
Hello world


In [9]:
import torch
data = torch.tensor(encode(text),dtype=torch.long)
print(data.shape,data.dtype)
print(data[:1000])

torch.Size([4176958]) torch.int64
tensor([47, 49, 48, 43, 37, 35, 28,  2,  2, 54, 72, 69, 82, 69,  9, 83,  2, 78,
        79, 84, 72, 73, 78, 71,  2, 84, 79,  2, 84, 69, 76, 76,  3,  2, 42, 69,
         9, 83,  2, 74, 85, 83, 84,  2, 83, 79, 77, 69,  2, 71, 85, 89,  2, 43,
         2, 87, 79, 82, 75,  2, 87, 73, 84, 72,  3,  1, 44, 49, 39, 59, 28,  2,
         2, 37,  9, 77, 79, 78, 14,  2, 89, 79, 85,  9, 82, 69,  2, 71, 79, 73,
        78, 71,  2, 79, 85, 84,  2, 87, 73, 84, 72,  2, 84, 72, 69,  2, 71, 85,
        89,  3,  2, 54, 72, 69, 82, 69,  9, 83,  2, 71, 79, 84, 84, 65,  2, 66,
        69,  2, 83, 79, 77, 69, 84, 72, 73, 78, 71,  2, 87, 82, 79, 78, 71,  2,
        87, 73, 84, 72,  2, 72, 73, 77,  3,  1, 37, 42, 35, 48, 38, 46, 39, 52,
        28,  2,  2, 53, 79,  2, 68, 79, 69, 83,  2, 72, 69,  2, 72, 65, 86, 69,
         2, 65,  2, 72, 85, 77, 80, 33,  2, 35,  2, 72, 85, 77, 80,  2, 65, 78,
        68,  2, 65,  2, 72, 65, 73, 82, 80, 73, 69, 67, 69, 33,  1, 50, 42, 49,
      

In [10]:
n = int(0.9*len(data))
train_data=data[:n]
val_data = data[n:]

In [11]:
torch.manual_seed(42)
batch_size = 4 # How much parallel computation happens at a time
block_size = 8 # Context length

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data)-block_size,(batch_size,))
  x = torch.stack([data[i:i+block_size]for i in ix])
  y= torch.stack([data[i+1:i+block_size+1]for i in ix])
  return x,y


In [12]:
xb,yb = get_batch('train')
print(xb)

tensor([[89,  2, 65, 82, 69,  2, 71, 79],
        [ 2,  2, 47, 69, 33,  2, 43,  9],
        [83,  2, 71, 79,  2, 84, 79,  2],
        [54, 79,  2, 52, 65, 67, 72, 69]])


In [13]:
import torch
import torch.nn as nn
from torch.nn import functional as F

torch.manual_seed(42)

class BigramLanguageModel(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)


  def forward(self,idx,targets=None):
    logits = self.token_embedding_table(idx)
    if targets == None:
      loss = None
    else:
      B, T,C = logits.shape
      logits = logits.view(B*T,C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits,targets)
    return logits,loss



  def generate(self,idx,max_new_tokens):
    for _ in range(max_new_tokens):
      logits,loss = self(idx)
      # Focus only on the last time step
      logits = logits[:,-1,:] # Becomes (B,C)
      probs = F.softmax(logits,dim = -1) # (B,C)
      idx_next = torch.multinomial(probs,num_samples = 1) # (B,1)
      idx = torch.cat((idx,idx_next),dim = 1) #(B,T+1)
    return idx


m = BigramLanguageModel(vocab_size)
logits , loss = m(xb,yb)
print(logits.shape)
print(loss)


print(decode(m.generate(torch.zeros((1,1),dtype = torch.long),max_new_tokens = 100)[0].tolist()))

torch.Size([32, 114])
tensor(5.0856, grad_fn=<NllLossBackward0>)
	.>­¦IYyY­|¦īw2”VK5P;JNBfNkA`OIWn|â)=1‘	éè1[wíä`rxoéèT?­Tj5Lä<	­!:’yI¦R’y W?i€€[èī™JMg”H�q5!1D.ävéçtl


In [14]:
optimizer = torch.optim.AdamW(m.parameters(),lr=1e-3)

In [15]:
batch_size = 32
for steps in range(10000):
  xb,yb = get_batch('train')
  logits , loss= m(xb,yb)
  optimizer.zero_grad(set_to_none = True)
  loss.backward()
  optimizer.step()
print(loss.item())

2.3311877250671387


In [16]:
print(decode(m.generate(torch.zeros((1,1),dtype = torch.long),max_new_tokens = 100)[0].tolist()))

	bonosmmouretit  hutha le met_DLER: pemed, simitesod'mere*43–qul—oweaschin8çxtot thenoup.
ROhay? myoo


In [19]:
# Self attention model

torch.manual_seed(42)

B,T,C =4,8,32
x = torch.randn(B,T,C)



# so for a single head
head_size = 16
key = nn.Linear(C,head_size,bias = False)
query = nn.Linear(C,head_size,bias = False)
value = nn.Linear(C,head_size, bias  = False)

k  = key(x) # (B,T,head_size)
q = query(x) # (B,T,head_size)

wei = q @ k.transpose(-1,-2) # (B, T,16) x (B,16,T) -> (B,T,T)
tril = torch.tril(torch.ones(T,T))

wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei,dim = -1)
v = value(x)
out = wei @ v # (B,T,16)
out.shape

torch.Size([4, 8, 16])

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F


# Hyper Parameters

batch_size = 64
block_size = 256
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2

torch.manual_seed(42)
print(device)

with open('Friends_script.txt','r',encoding='utf-8') as f:
    text = f.read()



chars = sorted(list(set(text)))
vocab_size = len(chars)


stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l:''.join([itos[i] for i in l])


data = torch.tensor(encode(text),dtype=torch.long)


n = int(0.9*len(data))
train_data=data[:n]
val_data = data[n:]



def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size,(batch_size,))
    x = torch.stack([data[i:i+block_size]for i in ix])
    y= torch.stack([data[i+1:i+block_size+1]for i in ix])
    return x,y


@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train','val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            X,Y = X.to(device), Y.to(device)
            logits,loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out





class Head(nn.Module):
    """ Implements one head of self attention """
    def __init__(self,head_size):
        super().__init__()
        self.key = nn.Linear(n_embd,head_size,bias=False)
        self.value = nn.Linear(n_embd,head_size,bias=False)
        self.query = nn.Linear(n_embd,head_size,bias=False)
        self.register_buffer('tril',torch.tril(torch.ones(block_size,block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self,x):
        B,T,C  = x.shape
        k = self.key(x)
        q = self.query(x)


        wei = q @ k.transpose(-1,-2) * C**(-0.5)
        wei = wei.masked_fill(self.tril[:T,:T]==0,float('-inf'))
        wei = F.softmax(wei,dim=  -1)
        wei = self.dropout(wei)


        v = self.value(x)
        out = wei @ v
        return out




class MultiHeadAttention(nn.Module):
    """ Multiple heads of self attention in parallel"""
    def __init__(self,num_heads,head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(num_heads*head_size,n_embd)
        self.dropout = nn.Dropout(dropout)


    def forward(self,x):
        out=  torch.cat([h(x) for h in self.heads],dim= -1)
        out = self.proj(out)
        out = self.dropout(out)
        return out




class FeedForward(nn.Module):
    """ Feed forward layer . Note -> Feed forward network works at a token level (independent)"""

    def __init__(self,n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd,n_embd),
            nn.Dropout(dropout),
        )

    def forward(self,x):
        return self.net(x)




class Block(nn.Module):
    """ Transformer block (decoder sec) : Communication followed by computation"""


    def __init__(self,n_embd,n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head,head_size=head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self,x):
        # x = self.sa(x)
        # x = self.ffwd(x)
        x = self.ln1(x)
        x = x+ self.sa(x)
        x = self.ln2(x)
        x = x + self.ffwd(x)


        return x





class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,n_embd)
        self.position_embedding_table = nn.Embedding(block_size,n_embd)
        # self.sa_heads = MultiHeadAttention(4,n_embd//4)  # Head -> 32 dim, now split into 4 sets of 8 dim
        # self.ffwd = FeedForward(n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd,n_head=n_head) for _ in range(n_layer)])
        self.lm_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd,vocab_size)


    def forward(self,idx,targets=None):


        B,T = idx.shape

        token_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T,device=device)) # (T,C)
        x = token_emb+pos_emb  # (B,T,C)
        # x = self.sa_heads(x)
        # x = self.ffwd(x)
        x = self.blocks(x)
        logits =self.lm_head(x)  # (B,T,vocab_size)
        if targets == None:
            loss = None
        else:
            B, T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits,targets)
        return logits,loss



    def generate(self,idx,max_new_tokens):
        for _ in range(max_new_tokens):
            # Crop idx to only the last block_size tokens
            idx_cond = idx[:,-block_size:]
            logits,loss = self(idx_cond)
            # Focus only on the last time step
            logits = logits[:,-1,:] # Becomes (B,C)
            probs = F.softmax(logits,dim = -1) # (B,C)
            idx_next = torch.multinomial(probs,num_samples = 1) # (B,1)
            idx = torch.cat((idx,idx_next),dim = 1) #(B,T+1)
        return idx


model = BigramLanguageModel()
m = model.to(device)

optimizer = torch.optim.AdamW(model.parameters(),lr=learning_rate)
for iter in range(max_iters):
    if iter % eval_interval == 0:
        losses = estimate_loss()
        print(f"Epoch : {iter} | train loss : {losses['train']:.4f} | val losses: {losses['val']:.4f}")


    xb,yb = get_batch('train')
    xb, yb = xb.to(device), yb.to(device)
    logits,loss = model(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


context = torch.zeros((1,1),dtype = torch.long,device = device)
print(decode(m.generate(context,max_new_tokens=500)[0].tolist()))






cuda
Epoch : 0 | train loss : 4.8632 | val losses: 4.8694
Epoch : 500 | train loss : 1.7259 | val losses: 1.7727
Epoch : 1000 | train loss : 1.4086 | val losses: 1.4732
Epoch : 1500 | train loss : 1.2860 | val losses: 1.3637
Epoch : 2000 | train loss : 1.2121 | val losses: 1.2948
Epoch : 2500 | train loss : 1.1668 | val losses: 1.2548
Epoch : 3000 | train loss : 1.1256 | val losses: 1.2204
Epoch : 3500 | train loss : 1.0986 | val losses: 1.2021
Epoch : 4000 | train loss : 1.0718 | val losses: 1.1732
Epoch : 4500 | train loss : 1.0509 | val losses: 1.1609
	al, always the water matake? As a week resquare it of matchia's fine.
ROSS:  I am so sorry...
PHOEBE:  All right, facaus...Ince we pick up when the entire with pictures out of my other cout enterpective, and senny, anyway, what about what he you did, like?
ROSS:  Long?
PHOEBE:  (to Rachel) Too. That’s now?
RACHEL:  That could’ve been worsed
PHOEBE:  You whould…I save it looks like final on you?!
RACHEL:  You lose him?
MONICA:  Ohh, I 

In [3]:
context = torch.zeros((1,1),dtype = torch.long,device = device)
print(decode(m.generate(context,max_new_tokens=1000)[0].tolist()))

	ing this thing I’m gonna call along, and if…there are scarastas in the bedreet don’t ex!
RACHEL:  Come tonight!
MANNY:  You said your cheese a largelet holding.
RACHEL:  Don’t just an isslack such Tpen them! But, now he misssed outside and-all then sopable of they to hold never next goody.
CHANDLER:  Wonder you instrading; those weeks in the dational light now.
MAN:  I get out to here, a new guy, dancer what-was.
ROSS:  I know know, I knew you and on hole that empty-pancy, because we said that last, at there again. Oh, look, um, I remember home, you know, it is napple either, and that's sore partying you doing the chick on your just tired, you et, and you didn't take that I'm not withing me, this guy, he loved and.... and no, tell you that white doing shot commes about I can't cry. You love to nap stips in cascar, okay?.. and we will come tocking on thus to I cantince, and Repost for or an elicial scrows...a fire sconwstick.
JANICE:  (clossing Carol) Stay Foldie Srogol.
JANICE:  Rexcu

In [6]:
context = torch.ones((1,1),dtype = torch.long,device = device)
print(decode(m.generate(context,max_new_tokens=1000)[0].tolist()))


JOEY:  You know, but then!! [Joey and Mona still looking at it!
PHOEBE:  I'll be a better! (Chandler does follows behind and he disgusting thing (points his room) I will, will you die and fell welling you with you, but I'll go over to give 25 out of you to get a face! (on the pause) Aw?!
ROSS:  I got this baby!
CHANDLER:  Y'know that been up, that works! I just left this cat!
ROSS:  Excuse me!
CHANDLER:  [to the going at head] Isn't that all manI for you?
ROB:  You are by Richards's storane who you're scarying lists…
DR. BREED BY:  By each! Don’t talk tomats!
CHANDLER:  Oh god! No well then, he'd tough that all my neighbon’s .
JOEY:  He's party!
RICHARD:  How hard pal.
DR. BREDDIE:  Sure, voice we're testae things all unounhtil lets you guys, and an one fancy pants
JOEY:  (looking at this) She's working at a salsocking direction! I think this boss yards off.
DR. BOWN:  (to check) Joey, I was talking about!!!
CHANDLER:  There are gonna be here?!
RACHEL:  The recerinisms she room. You s

In [5]:
# Save the model and optimizer state
model_save_path = "friends_language_model.pth"
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
}, model_save_path)
print(f"Model saved to {model_save_path}")


Model saved to friends_language_model.pth
